In [97]:
# pip install ISLP

In [143]:
#basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#preprossing imports
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder,RobustScaler,FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#model development imports
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA
from sklearn import tree
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, silhouette_score
from sklearn.cluster import KMeans , AgglomerativeClustering

from scipy.cluster.hierarchy import dendrogram, linkage

from ISLP import confusion_table as ct

In [99]:
data = pd.read_excel('/content/sample_data/Dry_Bean_Dataset.xlsx')

In [100]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [102]:
data.shape

(13611, 17)

In [103]:
null_values = data.isnull().sum()
null_values

Area               0
Perimeter          0
MajorAxisLength    0
MinorAxisLength    2
AspectRation       3
Eccentricity       0
ConvexArea         2
EquivDiameter      1
Extent             3
Solidity           3
roundness          0
Compactness        0
ShapeFactor1       1
ShapeFactor2       1
ShapeFactor3       1
ShapeFactor4       0
Class              0
dtype: int64

In [104]:
data = data.dropna()
data.shape

(13595, 17)

In [105]:
# data.info()
data.describe()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
count,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000
mean,53068.956822,855.508484,327.593180,202.286773,1.583700,0.751837,53789.334903,253.119798,0.749699,0.987655,0.873193,0.799728,0.006563,0.002303,0.643361,0.995059
std,29336.596904,214.314581,859.186428,44.993774,0.246454,0.121267,29787.506879,59.189571,0.049103,0.060326,0.059494,0.061618,0.001129,0.068600,0.098818,0.004367
min,9.000000,524.736000,183.601165,122.512653,1.024868,0.218951,9.000000,161.243764,0.555315,0.919246,0.489618,0.640577,0.002778,0.000564,0.410339,0.947687
25%,36363.500000,703.859000,253.426223,175.804945,1.432845,0.716207,36738.500000,215.173061,0.718571,0.985662,0.832019,0.762380,0.005899,0.001153,0.581223,0.993698
50%,44670.000000,795.123000,297.061200,192.452036,1.551481,0.764568,45222.000000,238.486080,0.759709,0.988281,0.883091,0.801157,0.006645,0.001692,0.641852,0.996382
75%,61348.500000,977.333500,376.520088,217.094129,1.707417,0.810550,62312.000000,279.484054,0.786858,0.990011,0.916807,0.834064,0.007272,0.002169,0.695663,0.997877
max,254616.000000,1985.370000,100000.000000,460.198497,2.430306,10.000000,263261.000000,569.374358,0.866195,8.000000,0.990685,0.987303,0.010451,8.000000,0.974767,0.999733


In [106]:
#Preprossing steps

In [107]:
#replacing class data with values
set(data['Class'])
encoder = LabelEncoder()
data['Class'] = encoder.fit_transform(data['Class'])
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715.0,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,5
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172.0,191.272751,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,5
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690.0,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,5
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724.0,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,5
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417.0,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,5


In [108]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'BARBUNYA',
 1: 'BOMBAY',
 2: 'CALI',
 3: 'DERMASON',
 4: 'HOROZ',
 5: 'SEKER',
 6: 'SIRA'}

In [109]:
# tried mim max scaler as well but not able to scale well since variance is very high
scaler = RobustScaler()
features = data.columns.drop(['Class' , 'Solidity' , 'ShapeFactor2' , 'ShapeFactor4' , 'ShapeFactor3'])
data[features] = scaler.fit_transform(data[features])

# to make data +ve
for col in data:
    if data[col].min() < 0:
        data[col] += abs(data[col].min())

data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,1.136122,0.312845,0.199660,1.244299,0.627607,3.506996,1.122490,0.449337,3.054860,0.988856,5.524476,3.805295,3.314448,0.003147,0.834222,0.998724,5
1,1.149690,0.414232,0.137486,1.458536,0.264006,2.043965,1.140360,0.466934,3.348409,0.984986,4.687169,4.370314,3.057614,0.003564,0.909851,0.998430,5
2,1.175545,0.363376,0.237420,1.293765,0.673210,3.643891,1.160615,0.500181,3.262670,0.989559,5.404439,3.741355,3.250689,0.003048,0.825871,0.999066,5
3,1.200680,0.442996,0.218994,1.453259,0.468985,2.964336,1.201048,0.532153,3.329564,0.976696,4.886522,4.014142,3.085325,0.003215,0.861794,0.994199,5
4,1.205964,0.348837,0.148234,1.641268,0.130859,1.216076,1.189043,0.538831,3.189227,0.990893,5.841147,4.602646,2.852604,0.003665,0.941900,0.999166,5


In [110]:
data.describe()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
count,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000,13595.000000
mean,2.123672,1.209519,1.169774,1.932083,2.035280,5.648384,2.102971,1.428621,2.846567,0.987655,4.523932,2.220155,2.755332,0.002303,0.643361,0.995059,3.530930
std,1.174168,0.783673,6.979929,1.089723,0.897591,1.285380,1.164780,0.920365,0.719071,0.060326,0.701675,0.859565,0.821516,0.068600,0.098818,0.004367,1.829091
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.919246,0.000000,0.000000,0.000000,0.000564,0.410339,0.947687,0.000000
25%,1.455053,0.654990,0.567251,1.290708,1.485861,5.270713,1.436233,0.838570,2.390729,0.985662,4.038320,1.699151,2.271449,0.001153,0.581223,0.993698,2.000000
50%,1.787513,0.988710,0.921736,1.693891,1.917936,5.783324,1.767963,1.201075,2.993153,0.988281,4.640667,2.240094,2.814666,0.001692,0.641852,0.996382,3.000000
75%,2.455053,1.654990,1.567251,2.290708,2.485861,6.270713,2.436233,1.838570,3.390729,0.990011,5.038320,2.699151,3.271449,0.002169,0.695663,0.997877,5.000000
max,10.190394,5.341024,810.896617,8.178554,5.118638,103.675284,10.293937,6.346203,4.552537,8.000000,5.909652,4.836831,5.585224,8.000000,0.974767,0.999733,6.000000


In [111]:
# plt.scatter(data['Area'] ,data['Class'] )

In [112]:
data.columns

Index(['Area', 'Perimeter', 'MajorAxisLength', 'MinorAxisLength',
       'AspectRation', 'Eccentricity', 'ConvexArea', 'EquivDiameter', 'Extent',
       'Solidity', 'roundness', 'Compactness', 'ShapeFactor1', 'ShapeFactor2',
       'ShapeFactor3', 'ShapeFactor4', 'Class'],
      dtype='object')

In [113]:
# selected_features = ['Extent',
#        'Solidity', 'roundness', 'Compactness', 'ShapeFactor1','Class']
# sub_Data = data[selected_features]
# type(sub_Data)
# sns.pairplot(sub_Data, hue='Class', kind='scatter') # very cost heavy process
# data


# df_long = data.melt(id_vars=['Class'], var_name='Feature')
# df_long
# Plot categorical scatter plot of each feature against 'Class'
# sns.catplot(data=df_long, x='Class', y='value', hue='Feature', kind='swarm') # cost heavy operation


# pred = data[data.columns.drop(['Class'])]
# label = data['Class']
# type(pred) # may be i should do after some steps

In [114]:
#outliers
orf = ['ShapeFactor2','MajorAxisLength', 'Eccentricity','Solidity'] # outliers_removal_features , i plotted scatter to see weather outliers are present or not
z_scores = (data[orf] - data[orf].mean()) / data[orf].std()
limiter = 5
data = data[(z_scores.abs() < limiter).all(axis=1)]
data.describe()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
count,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000
mean,2.123876,1.209689,1.110357,1.932135,2.035692,5.641686,2.103174,1.428803,2.846477,0.987139,4.523748,2.219690,2.755337,0.001714,0.643305,0.995058,3.530498
std,1.174281,0.783724,0.696067,1.089878,0.897398,0.971771,1.164891,0.920438,0.719149,0.004662,0.701655,0.859258,0.821635,0.000594,0.098778,0.004367,1.829187
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.919246,0.000000,0.000000,0.000000,0.000564,0.410339,0.947687,0.000000
25%,1.455193,0.655081,0.567372,1.290400,1.486383,5.271185,1.436370,0.838731,2.390086,0.985659,4.038272,1.698961,2.271153,0.001153,0.581202,0.993695,2.000000
50%,1.787673,0.988970,0.921888,1.693891,1.918038,5.783405,1.768041,1.201241,2.992827,0.988281,4.640560,2.239828,2.814721,0.001692,0.641822,0.996380,3.000000
75%,2.455193,1.655089,1.567251,2.291433,2.485969,6.270713,2.436800,1.838694,3.390729,0.990011,5.038159,2.698390,3.271619,0.002168,0.695572,0.997875,5.000000
max,10.190394,5.341024,4.510858,8.178554,5.118638,7.339929,10.293937,6.346203,4.552537,0.994677,5.909652,4.836831,5.585224,0.003665,0.974767,0.999733,6.000000


In [115]:
data.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,1.136122,0.312845,0.199660,1.244299,0.627607,3.506996,1.122490,0.449337,3.054860,0.988856,5.524476,3.805295,3.314448,0.003147,0.834222,0.998724,5
1,1.149690,0.414232,0.137486,1.458536,0.264006,2.043965,1.140360,0.466934,3.348409,0.984986,4.687169,4.370314,3.057614,0.003564,0.909851,0.998430,5
2,1.175545,0.363376,0.237420,1.293765,0.673210,3.643891,1.160615,0.500181,3.262670,0.989559,5.404439,3.741355,3.250689,0.003048,0.825871,0.999066,5
3,1.200680,0.442996,0.218994,1.453259,0.468985,2.964336,1.201048,0.532153,3.329564,0.976696,4.886522,4.014142,3.085325,0.003215,0.861794,0.994199,5
4,1.205964,0.348837,0.148234,1.641268,0.130859,1.216076,1.189043,0.538831,3.189227,0.990893,5.841147,4.602646,2.852604,0.003665,0.941900,0.999166,5


In [116]:
# plt.scatter(data['Eccentricity'] ,data['Class'] )

In [117]:
# important block generates all plots vs graph

# features = data.columns[:-1]

# for feature in features:
#     plt.figure(figsize=(8, 6))
#     sns.scatterplot(x=feature, y='Class', data=data, alpha=0.5)
#     # sns.lineplot(x=feature, y='Class', data=data.groupby(feature)['Class'].mean(), color='red')
#     plt.title(f'Scatter and Line Plot for {feature} vs. Class')
#     plt.xlabel(feature)
#     plt.ylabel('Class')
#     plt.grid(True)



In [118]:
# sns.pairplot(data, hue='Class', diag_kind='kde') # very cost heavy process

In [119]:
data.head(1)

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,1.136122,0.312845,0.19966,1.244299,0.627607,3.506996,1.12249,0.449337,3.05486,0.988856,5.524476,3.805295,3.314448,0.003147,0.834222,0.998724,5


In [120]:
# bad way to edit data
# names=['AR/ECC', 'AR/PE/MA/ED/CA', 'CO/SF3']
# features = ['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3']


# combined_feature = data[['AspectRation' ,'Eccentricity' ]].mean(axis = 1)
# data['Ar/Ecc'] = combined_feature

# combined_feature2 = data[[ 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea' ]].mean(axis = 1)
# data['AR/PE/MA/ED/CA'] = combined_feature2

# combined_feature3 = data[['Compactness', 'ShapeFactor3' ]].mean(axis = 1)
# data['CO/SF3'] = combined_feature3

# data.drop(['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3'] , axis=1,inplace=True)
# data.head()


# bad way to modify

# # Define the combine_features function
# def combine_features(X):
#     combined_features = []
#     combined_features.append(X[['AspectRation', 'Eccentricity']].mean(axis=1))
#     combined_features.append(X[['Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea']].mean(axis=1))
#     combined_features.append(X[['Compactness', 'ShapeFactor3']].mean(axis=1))
#     return pd.concat(combined_features, axis=1)

# names=['AR/ECC', 'AR/PE/MA/ED/CA', 'CO/SF3']

# # Define the combined_feature_transformer
# combined_feature_transformer = ColumnTransformer(
#     transformers=[
#         ('combine', FunctionTransformer(validate=False), ['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3'])
#     ],
#     remainder='passthrough'
# )

# # Define the pipeline
# pipeline = Pipeline([
#     ('combine_features', combined_feature_transformer)
# ])

# # Fit and transform the pipeline
# combined_features = pipeline.fit_transform(df)


# df_combined = pd.DataFrame(combined_features, index=df.index, names=['AR/ECC', 'AR/PE/MA/ED/CA', 'CO/SF3'])
# print(df_combined)


In [121]:
# data2 = data.copy()
features = ['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3']
names = ['AR/ECC', 'AR/PE/MA/ED/CA', 'CO/SF3']


combined_features_df = pd.DataFrame()


for name, feature_group in zip(names, [features[:2], features[2:7], features[7:]]):
    combined_feature = data[feature_group].mean(axis=1)
    combined_features_df[name] = combined_feature

combined_features_df

data = pd.concat([data , pd.DataFrame(combined_features_df)] , axis = 1 )
data.drop(['AspectRation', 'Eccentricity', 'Area', 'Perimeter', 'MajorAxisLength', 'EquivDiameter', 'ConvexArea', 'Compactness', 'ShapeFactor3'] , axis=1,inplace=True)
data.head()

,MinorAxisLength,Extent,Solidity,roundness,ShapeFactor1,ShapeFactor2,ShapeFactor4,Class,AR/ECC,AR/PE/MA/ED/CA,CO/SF3
0,1.244299,3.054860,0.988856,5.524476,3.314448,0.003147,0.998724,5,2.067302,0.644091,2.319759
1,1.458536,3.348409,0.984986,4.687169,3.057614,0.003564,0.998430,5,1.153985,0.661740,2.640082
2,1.293765,3.262670,0.989559,5.404439,3.250689,0.003048,0.999066,5,2.158551,0.687428,2.283613
3,1.453259,3.329564,0.976696,4.886522,3.085325,0.003215,0.994199,5,1.716661,0.719174,2.437968
4,1.641268,3.189227,0.990893,5.841147,2.852604,0.003665,0.999166,5,0.673468,0.686182,2.772273


In [122]:
#to shift Y(class) to right
col = data.pop('Class')
data['Class'] = col
# data.head(1)
data.describe()

,MinorAxisLength,Extent,Solidity,roundness,ShapeFactor1,ShapeFactor2,ShapeFactor4,AR/ECC,AR/PE/MA/ED/CA,CO/SF3,Class
count,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000,13591.000000
mean,1.932135,2.846477,0.987139,4.523748,2.755337,0.001714,0.995058,3.838689,1.595180,1.431498,3.530498
std,1.089878,0.719149,0.004662,0.701655,0.821635,0.000594,0.004367,0.916776,0.937445,0.478960,1.829187
min,0.000000,0.000000,0.919246,0.000000,0.000000,0.000564,0.947687,0.000000,0.325220,0.205169,0.000000
25%,1.290400,2.390086,0.985659,4.038272,2.271153,0.001153,0.993695,3.378784,0.991421,1.140081,2.000000
50%,1.693891,2.992827,0.988281,4.640560,2.814721,0.001692,0.996380,3.850722,1.330869,1.440825,3.000000
75%,2.291433,3.390729,0.990011,5.038159,3.271619,0.002168,0.997875,4.378341,1.993425,1.696981,5.000000
max,8.178554,4.552537,0.994677,5.909652,5.585224,0.003665,0.999733,6.229284,7.336483,2.905799,6.000000


##<------------------------------ Preprocessing Part Done ----------------------------->


Certainly! Here's the breakdown:

## Using scikit-learn (sklearn):

Supervised methods:
- Logistic Regression
- KNN (K-Nearest Neighbors)
- LDA (Linear Discriminant Analysis)
- QDA (Quadratic Discriminant Analysis)
- Decision Trees
- Naive Bayes

Unsupervised methods:
- K-Means Clustering
- Hierarchical Clustering

## Using TensorFlow (tf):

Supervised methods:

- SVM (Support Vector Machines)
- Neural Network

In [123]:
#calculating no of features
for i in range(7):
  print(i ,'count :' ,(data['Class'] == i).sum() )

0 count : 1322
1 count : 522
2 count : 1630
3 count : 3546
4 count : 1928
5 count : 2007
6 count : 2636


In [124]:
X = data[data.columns.drop('Class')]
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Logistic Regression

In [125]:
#100 iteration gives same accuracy as 1lakh iterations
logistic_reg = LogisticRegression(max_iter=100, multi_class='multinomial', solver='lbfgs')


logistic_reg.fit(X_train, y_train)
y_pred = logistic_reg.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
# print(conf_matrix)
ct(y_pred,y_test) # islp package shows good confusion table

Accuracy: 0.9124678190511217
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.86      0.89       270
           1       0.99      1.00      1.00       117
           2       0.92      0.92      0.92       319
           3       0.91      0.90      0.90       671
           4       0.95      0.95      0.95       377
           5       0.95      0.93      0.94       406
           6       0.85      0.89      0.87       559

    accuracy                           0.91      2719
   macro avg       0.93      0.92      0.92      2719
weighted avg       0.91      0.91      0.91      2719

Confusion Matrix:


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Truth,0,1,2,3,4,5,6
Predicted,,,,,,,
0,233,0,17,0,0,4,0
1,1,117,0,0,0,0,0
2,19,0,295,0,7,0,0
3,0,0,0,603,4,13,43
4,3,0,2,1,358,0,14
5,2,0,1,11,0,379,6
6,12,0,4,56,8,10,496


KNN

In [126]:
no_of_neigh = 15 # no of neighbours
knn = KNeighborsClassifier(n_neighbors=no_of_neigh)
knn.fit(X_train , y_train)
y_pred_knn = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Classification Report:")
print(classification_report(y_test, y_pred_knn))

ct(y_test , y_pred_knn)

Accuracy: 0.917616770871644
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.84      0.88       270
           1       0.99      1.00      1.00       117
           2       0.90      0.94      0.92       319
           3       0.91      0.91      0.91       671
           4       0.95      0.95      0.95       377
           5       0.96      0.95      0.96       406
           6       0.86      0.89      0.88       559

    accuracy                           0.92      2719
   macro avg       0.93      0.93      0.93      2719
weighted avg       0.92      0.92      0.92      2719



Truth,0,1,2,3,4,5,6
Predicted,,,,,,,
0,227,1,26,0,3,2,11
1,0,117,0,0,0,0,0
2,11,0,299,0,2,1,6
3,0,0,0,608,2,11,50
4,2,0,7,2,357,0,9
5,2,0,0,10,0,387,7
6,1,0,1,45,10,2,500


Linear Discriminant Analysis

In [127]:
clf = LDA()
clf.fit(X_train, y_train)
y_pred_lda = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_lda))
print("Classification Report:")
print(classification_report(y_test, y_pred_lda))

ct(y_test , y_pred_lda)

Accuracy: 0.9014343508642884
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.84      0.89       270
           1       0.99      1.00      1.00       117
           2       0.93      0.93      0.93       319
           3       0.94      0.83      0.88       671
           4       0.96      0.94      0.95       377
           5       0.97      0.90      0.94       406
           6       0.76      0.94      0.84       559

    accuracy                           0.90      2719
   macro avg       0.93      0.91      0.92      2719
weighted avg       0.91      0.90      0.90      2719



Truth,0,1,2,3,4,5,6
Predicted,,,,,,,
0,227,1,17,0,3,1,21
1,0,117,0,0,0,0,0
2,8,0,298,0,3,1,9
3,0,0,0,559,0,9,103
4,1,0,6,4,356,0,10
5,5,0,0,8,0,367,26
6,0,0,1,23,7,1,527


In [128]:
qlf = QDA()
qlf.fit(X_train, y_train)
y_pred_qda = qlf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_qda))
print("Classification Report:")
print(classification_report(y_test, y_pred_qda))

ct(y_test , y_pred_qda)

Accuracy: 0.9084222140492828
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.86      0.90       270
           1       0.99      1.00      1.00       117
           2       0.90      0.95      0.93       319
           3       0.94      0.83      0.88       671
           4       0.95      0.96      0.96       377
           5       0.96      0.94      0.95       406
           6       0.79      0.93      0.86       559

    accuracy                           0.91      2719
   macro avg       0.93      0.92      0.92      2719
weighted avg       0.91      0.91      0.91      2719



/usr/local/lib/python3.10/dist-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Truth,0,1,2,3,4,5,6
Predicted,,,,,,,
0,231,1,23,0,3,4,8
1,0,117,0,0,0,0,0
2,8,0,304,0,4,1,2
3,0,0,0,555,1,9,106
4,1,0,7,2,363,0,4
5,1,0,0,9,0,382,14
6,1,0,3,24,11,2,518


Decision Trees

In [129]:
decision_tree = tree.DecisionTreeClassifier()
decision_tree.fit(X_train , y_train)
y_pred_decTree = decision_tree.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred_decTree))
print("Classification Report:")
print(classification_report(y_test, y_pred_decTree))

ct(y_test , y_pred_decTree)

Accuracy: 0.8948142699521883
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.87      0.89       270
           1       0.99      1.00      1.00       117
           2       0.91      0.90      0.91       319
           3       0.89      0.88      0.89       671
           4       0.91      0.95      0.93       377
           5       0.93      0.92      0.92       406
           6       0.83      0.84      0.83       559

    accuracy                           0.89      2719
   macro avg       0.91      0.91      0.91      2719
weighted avg       0.89      0.89      0.89      2719



Truth,0,1,2,3,4,5,6
Predicted,,,,,,,
0,236,1,15,0,4,5,9
1,0,117,0,0,0,0,0
2,17,0,288,0,10,1,3
3,0,0,0,593,4,7,67
4,1,0,9,2,359,0,6
5,2,0,2,16,0,372,14
6,6,0,3,52,16,14,468


Naive Bayes

In [130]:
naive_model = MNB()
naive_model.fit(X_train,y_train)
y_pred_nb = naive_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("Classification Report:")
print(classification_report(y_test, y_pred_nb))

ct(y_test , y_pred_nb)

Accuracy: 0.7716072085325487
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.25      0.40       270
           1       0.99      1.00      1.00       117
           2       0.67      0.87      0.76       319
           3       0.69      1.00      0.81       671
           4       0.98      0.84      0.90       377
           5       0.98      0.72      0.83       406
           6       0.68      0.65      0.66       559

    accuracy                           0.77      2719
   macro avg       0.86      0.76      0.77      2719
weighted avg       0.81      0.77      0.76      2719



Truth,0,1,2,3,4,5,6
Predicted,,,,,,,
0,67,1,133,0,2,2,65
1,0,117,0,0,0,0,0
2,0,0,277,0,2,1,39
3,0,0,0,668,0,3,0
4,0,0,2,20,315,0,40
5,0,0,0,88,0,292,26
6,0,0,0,194,3,0,362


## Clustering

k-means

In [131]:
kmeans = KMeans(n_clusters=7, random_state=33, n_init="auto").fit(X_train)
kmeans_labels = kmeans.predict(X_test)


In [132]:
# set(kmeans.labels_)
for i in range(7):
  print(i ,'count :' ,(kmeans_labels == i).sum() )

0 count : 425
1 count : 118
2 count : 563
3 count : 455
4 count : 367
5 count : 393
6 count : 398


In [133]:
for i in range(7):
  print(i ,'count :' ,(y_test == i).sum() )

0 count : 270
1 count : 117
2 count : 319
3 count : 671
4 count : 377
5 count : 406
6 count : 559


In [134]:
#ranges between -1 and 1 i.e .35 score suggests that clusteres are indeed well seperted
silhouette = silhouette_score(X_test, kmeans_labels)
print("Silhouette Score:", silhouette)

Silhouette Score: 0.3509856979958714


GPT interperetation

A silhouette score of 0.35 suggests that the clusters in your data are reasonably well-separated. Here's how to interpret the silhouette score:

The silhouette score ranges from -1 to 1.
A score close to 1 indicates that the data points are well-clustered, with each point being closer to its own cluster than to other clusters.
A score close to 0 suggests overlapping clusters or clusters with similar densities.
A negative score indicates that the data points might have been assigned to the wrong clusters.
In your case, a silhouette score of 0.35 indicates that the clusters are somewhat well-separated, but there may still be some degree of overlap or suboptimal clustering. It's generally a decent score, but it's always essential to consider the context of your specific dataset and problem domain when interpreting clustering results. Additionally, visual inspection of the clusters can provide more insights into their quality and structure.

In [135]:
data_int = X_test.copy() # created for data interperetation
data_int['kmeans_res'] = kmeans_labels
data_int['actual_res'] = y_test
# data_int[data_int['actual_res'] == 2].head(30)
# data_int[data_int['kmeans_res'] == 5].head(30)
data_int.head()

,MinorAxisLength,Extent,Solidity,roundness,ShapeFactor1,ShapeFactor2,ShapeFactor4,AR/ECC,AR/PE/MA/ED/CA,CO/SF3,kmeans_res,actual_res
1081,1.936251,2.861130,0.991197,5.476926,2.561175,0.002564,0.998768,2.293789,1.074249,2.225479,6,5
3901,2.210486,3.861489,0.991806,4.728041,2.328946,0.001410,0.996353,4.062256,1.793225,1.326470,5,2
3583,6.580399,3.291272,0.987128,4.942300,0.335439,0.001029,0.997217,3.182270,4.960784,1.800269,1,1
7295,1.719742,4.201317,0.989005,3.391154,2.807979,0.000830,0.991514,5.705705,2.165420,0.437166,4,4
6674,1.631869,2.995655,0.988501,4.040198,2.882211,0.001060,0.995164,5.127915,1.804769,0.741486,4,4


clustering interperetation its almost near to the y_test considering no labels are provided

table map

| y_actual | Clustering_res |
|----------|----------------|
|    0     |       2        |
|    1     |       1        |
|    2     |       2        |
|    3     |       3        |
|    4     |       4        |
|    5     |       6        |
|    6     |       0        |
|  2.6.4.3 |       5        |


In [136]:
#clustering interperetation its almost near to the y_test considering no labels are provided
'''
    y_actual    Clustering_res
    0           2
    1           1
    2           2
    3           3
    4           4
    5           6
    6           0
    2.6.4.3     5
'''

'\n    y_actual    Clustering_res\n    0           2\n    1           1\n    2           2\n    3           3\n    4           4\n    5           6\n    6           0\n    2.6.4.3     5\n'

Herarcial Clustering

In [137]:
n_clusters = 7
clustering = AgglomerativeClustering(n_clusters=n_clusters).fit(X_train)


cluster_labels = clustering.labels_
print("Cluster labels:", cluster_labels)


Cluster labels: [4 4 5 ... 1 4 3]


In [138]:
test_labels = clustering.fit_predict(X_test)
data_int['Hi_res'] = test_labels

In [139]:
#it seems for this data kmeans performs better though both methods provides good cluster seperation
data_int[data_int['Hi_res'] == 4].head(10)

,MinorAxisLength,Extent,Solidity,roundness,ShapeFactor1,ShapeFactor2,ShapeFactor4,AR/ECC,AR/PE/MA/ED/CA,CO/SF3,kmeans_res,actual_res,Hi_res
1081,1.936251,2.861130,0.991197,5.476926,2.561175,0.002564,0.998768,2.293789,1.074249,2.225479,6,5,4
370,1.640671,2.940119,0.987063,5.059754,2.855192,0.002629,0.998765,2.443425,0.941460,2.160053,6,5,4
12848,1.517461,2.936616,0.990286,5.183879,2.991789,0.002336,0.998248,2.973200,0.976084,1.908893,6,3,4
1340,1.884228,2.745160,0.992745,5.444799,2.610466,0.002272,0.998737,2.798174,1.153594,1.995844,6,5,4
110,1.628103,3.568083,0.988843,5.083841,2.866750,0.002972,0.999129,1.932960,0.844160,2.374566,6,5,4
1013,1.989329,2.957972,0.992980,5.751915,2.508084,0.002702,0.999652,2.024687,1.048415,2.340253,6,5,4
1646,2.135023,2.841053,0.989864,5.337869,2.384635,0.002333,0.998311,2.510157,1.243165,2.128542,6,5,4
1217,1.817670,3.359704,0.991914,5.412982,2.676879,0.002268,0.998318,2.851046,1.125704,1.968931,6,5,4
536,1.714387,3.280804,0.988598,5.345510,2.779704,0.002602,0.998493,2.421219,0.972796,2.168949,6,5,4
195,1.649752,3.726643,0.992887,5.637459,2.849456,0.002829,0.997974,2.128371,0.873669,2.292155,6,5,4


In [140]:
for i in range(7):
  print(i ,'count :' ,(test_labels == i).sum() )

0 count : 960
1 count : 573
2 count : 324
3 count : 238
4 count : 387
5 count : 118
6 count : 119


In [141]:
silhouette_hc = silhouette_score(X_test, test_labels)
silhouette_hc

0.337199501545768

In [142]:
#generating dendogram is cost heavy

# Z = linkage(X_train, method='ward')

# # Plot the dendrogram
# plt.figure(figsize=(2, 1))
# dendrogram(Z)
# plt.title('Hierarchical Clustering Dendrogram')
# plt.xlabel('Sample Index')
# plt.ylabel('Distance')
# plt.show()


<------------------- Most interperation using sklearn done ----------------->